In [3]:
import os
import pickle
import random
import gensim
from tqdm import tqdm
from IPython.display import clear_output
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
import wandb

# DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

wandb: https://wandb.ai/maximkiselev8376/aaa%20w2v%20experiment?nw=nwusermaximkiselev8376 (там нет особо ничего интересного, только файлики моделей)

In [4]:
# логинимся в вандбе
wandb.login(key='37d77515db6a159f86f6c7c987b7dc0eca96f53e')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
RANDOM_SEED = 42
TARGET = 'attr_value_name'

In [6]:
def set_seed(seed: int):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

# Подгружаем данные

In [7]:
df = pd.read_csv('/kaggle/input/final-project-data/data.csv')
df_splitting = pd.read_csv('/kaggle/input/final-project-data/splitting_sample.csv')
df_baseline = pd.read_csv('/kaggle/input/final-project-data/baseline_prediction.csv')
df_tfidf = pd.read_csv('/kaggle/input/final-project-data/tfidf_lr_prediction.csv')

df = df.merge(df_splitting, on='item_id', how='left')
df = df.merge(df_baseline, on='item_id', how='left')
df = df.merge(df_tfidf, on='item_id', how='left')

## Отделяем квартиры от домов

In [8]:
SUBSTRINGS_TO_REPLACE = [',', ' в', ' и', ' га', '-студия', '-студии']
WORDS_TO_DELETE = ['на', 'участке', 'доля', 'аукцион:']
SUBWORD_TO_DELETE = ['м²', 'сот.', '-к'] + [str(i) for i in range(10)]

def clear_description(descr: str):
    for substr in SUBSTRINGS_TO_REPLACE:
        descr = descr.replace(substr, '')
    descr = descr.lower().split(' ')
    result = []

    for word in descr:
        cnt_bad_subwords = 0
        for stop_subwords in SUBWORD_TO_DELETE:
            if stop_subwords in word:
                cnt_bad_subwords += 1
        if (word not in WORDS_TO_DELETE) and (cnt_bad_subwords == 0):
            result.append(word)
    
    return ' '.join(result)


descr_train = df.loc[df['sample_part'] == 'train', 'title'].apply(lambda descr: clear_description(descr))
descr_train.value_counts()

title
квартира            86623
дом                 23645
дача                 4366
коттедж              2194
апартаменты          1792
таунхаус              593
своб. планировка      195
квартире              195
своб. планировке        3
Name: count, dtype: int64

In [9]:
def classification_adv(descr: str):
    clear_desc = clear_description(descr)
    if clear_desc in {'квартира', 'квартире', 'своб. планировка', 'своб. планировке'}:
        return 'квартира'
    elif clear_desc in {'апартаменты'}:
        return 'апартаменты'
    elif clear_desc in {'дом'}:
        return 'дом'
    elif clear_desc in {'дача'}:
        return 'дача'
    elif clear_desc in {'коттедж'}:
        return 'коттедж'
    elif clear_desc in {'таунхаус'}:
        return 'таунхаус'
    
df['object_type'] = df['title'].apply(lambda descr: classification_adv(descr))
df['object_type'].value_counts()

object_type
квартира       145114
дом             39422
дача             7253
коттедж          3605
апартаменты      2980
таунхаус          970
Name: count, dtype: int64

## Создаем словарь

In [10]:
vectorizer = CountVectorizer(
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=10, # ставим ограничение на минимальное количество встреч в корпусе
    max_features=None
)
vectorizer.fit(df.loc[df['sample_part'] == 'train', 'description'])

# создаем токенайзер
tokenizer = vectorizer.build_tokenizer()
# создаем энкодер
encode_mapping = {word: i+1 for i, word in enumerate(vectorizer.vocabulary_)}

# Word2Vec

In [11]:
def custom_preprocess_sentence(sentence: str):
    tokens = tokenizer(sentence)
    return [token.lower() for token in tokens]
#     return [encode_mapping.get(token, 0) for token in tokens]

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""
    def __init__(self, df: pd.DataFrame, sample_part: str):
        self.df = df
        self.sample_part = sample_part
    
    def __iter__(self):
        for sentence in self.df.loc[self.df['sample_part'] == self.sample_part, 'description'].values:
            yield custom_preprocess_sentence(sentence)

In [12]:
train_sentences = MyCorpus(df=df, sample_part='train')
val_sentences = MyCorpus(df=df, sample_part='val')
test_sentences = MyCorpus(df=df, sample_part='test')

In [13]:
class log_callback(gensim.models.callbacks.CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print('Loss after epoch {}: {}'.format(self.epoch, loss))
        self.epoch += 1

## CBOW

In [14]:
cbow_model_1 = gensim.models.Word2Vec(
    sentences=train_sentences,
    vector_size=128,
    alpha=0.025,
    window=5,
    min_count=10,
    seed=RANDOM_SEED,
    workers=16,
    min_alpha=0.0001,
    sg=0,
    hs=0,
    cbow_mean=1,
    compute_loss=True,
    callbacks=[log_callback()],
    epochs=50,
    null_word=0
)

Loss after epoch 0: 935722.8125
Loss after epoch 1: 1555981.625
Loss after epoch 2: 2124699.0
Loss after epoch 3: 2650076.25
Loss after epoch 4: 3174896.5
Loss after epoch 5: 3673914.0
Loss after epoch 6: 4214544.5
Loss after epoch 7: 4710471.5
Loss after epoch 8: 5244750.0
Loss after epoch 9: 5748410.5
Loss after epoch 10: 6243341.0
Loss after epoch 11: 6729667.5
Loss after epoch 12: 7231411.5
Loss after epoch 13: 7734211.5
Loss after epoch 14: 8232154.0
Loss after epoch 15: 8710144.0
Loss after epoch 16: 9163112.0
Loss after epoch 17: 9595803.0
Loss after epoch 18: 10042910.0
Loss after epoch 19: 10468817.0
Loss after epoch 20: 10917441.0
Loss after epoch 21: 11359031.0
Loss after epoch 22: 11775829.0
Loss after epoch 23: 12219218.0
Loss after epoch 24: 12633108.0
Loss after epoch 25: 13095778.0
Loss after epoch 26: 13523403.0
Loss after epoch 27: 13926082.0
Loss after epoch 28: 14351103.0
Loss after epoch 29: 14755087.0
Loss after epoch 30: 15164532.0
Loss after epoch 31: 15592312.0

In [15]:
for item in train_sentences:
    break
item[:5]

['отличный', 'двухэтажный', 'коттедж', 'живописном', 'месте']

In [16]:
cbow_model_1.wv.similar_by_key('отличный', topn=10)

[('идеальный', 0.8582980632781982),
 ('прекрасный', 0.8525403738021851),
 ('хороший', 0.8020535707473755),
 ('шикарный', 0.7856317162513733),
 ('великолепный', 0.6998453140258789),
 ('замечательный', 0.6966826319694519),
 ('обменный', 0.6368477940559387),
 ('достойный', 0.6342856287956238),
 ('лучший', 0.6291736364364624),
 ('бабушкин', 0.6232607960700989)]

In [17]:
cbow_model_1.wv.similar_by_key('коттедж', topn=10)

[('таунхаус', 0.7924350500106812),
 ('особняк', 0.7353450059890747),
 ('дом', 0.6924506425857544),
 ('двухэтажный', 0.6346051692962646),
 ('котедж', 0.628898024559021),
 ('одноэтажный', 0.6286252737045288),
 ('загородный', 0.6144553422927856),
 ('дуплекс', 0.6073756217956543),
 ('квадрохаус', 0.5975426435470581),
 ('танхаус', 0.5973822474479675)]

### Логрег на выходах CBOW

In [23]:
def get_avg_vector(sentence: str, w2vmodel: gensim.models.Word2Vec):
    """Функция для получения векторов W2V и усреднения их по всем токенам.
    """
    preprocessed_sentence = custom_preprocess_sentence(sentence)
    vectors = [w2vmodel.wv.get_vector(key=word) for word in preprocessed_sentence if word in w2vmodel.wv]
    if not vectors:
        return list(np.zeros(w2vmodel.vector_size))
    return list(np.mean(vectors, axis=0))

In [24]:
df['cbow_1_vector'] = df['description'].apply(lambda sent: get_avg_vector(sent, cbow_model_1))

In [33]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(n_jobs=-1, max_iter=500, random_state=RANDOM_SEED))
])

pipe.fit(
    X=np.array(df.loc[df['sample_part'] == 'train', 'cbow_1_vector'].tolist()),
    y=df.loc[df['sample_part'] == 'train', TARGET].values
)

Pipeline(steps=[('scaler', StandardScaler()),
                ('lr',
                 LogisticRegression(max_iter=500, n_jobs=-1, random_state=42))])

In [35]:
df['cbow_1_lr_predict'] = pipe.predict(np.array(df['cbow_1_vector'].tolist()))

Смотрим качество

In [44]:
accuracy_score(
    y_true=df.loc[df['sample_part'] == 'test', TARGET],
    y_pred=df.loc[df['sample_part'] == 'test', 'cbow_1_lr_predict']
)

0.584316558311261

In [50]:
pickle.dump(pipe, open('/kaggle/working/cbow_1_lr.pkl', 'wb'))

In [52]:
run = wandb.init(project='aaa w2v experiment')
artifact = wandb.Artifact(name='cbow_1_logreg', type='data')
artifact.add_file('/kaggle/working/cbow_1_lr.pkl')
run.log_artifact(artifact)
run.finish()

Работает довольно плохо((

### Попробуем обучить бустинг

In [55]:
BASE_CATBOOST_PARAMS = {'random_seed': RANDOM_SEED, 'thread_count': -1}

In [59]:
# сначала фитим катбуст с дефолтными параметрами
clf_1 = CatBoostClassifier(**BASE_CATBOOST_PARAMS, verbose=True)

clf_1.fit(
    X=np.array(df.loc[df['sample_part'] == 'train', 'cbow_1_vector'].tolist()),
    y=df.loc[df['sample_part'] == 'train', TARGET].values,
    verbose=True
)

Learning rate set to 0.100992
0:	learn: 1.3385152	total: 426ms	remaining: 7m 5s
1:	learn: 1.3005844	total: 749ms	remaining: 6m 13s
2:	learn: 1.2688596	total: 1.08s	remaining: 5m 59s
3:	learn: 1.2421335	total: 1.39s	remaining: 5m 47s
4:	learn: 1.2192168	total: 1.73s	remaining: 5m 44s
5:	learn: 1.1992167	total: 2.08s	remaining: 5m 44s
6:	learn: 1.1829242	total: 2.4s	remaining: 5m 40s
7:	learn: 1.1676232	total: 2.74s	remaining: 5m 39s
8:	learn: 1.1543452	total: 3.06s	remaining: 5m 37s
9:	learn: 1.1421466	total: 3.39s	remaining: 5m 35s
10:	learn: 1.1314864	total: 3.68s	remaining: 5m 31s
11:	learn: 1.1222408	total: 4.02s	remaining: 5m 31s
12:	learn: 1.1144269	total: 4.33s	remaining: 5m 29s
13:	learn: 1.1062159	total: 4.68s	remaining: 5m 29s
14:	learn: 1.0993116	total: 4.96s	remaining: 5m 25s
15:	learn: 1.0928914	total: 5.26s	remaining: 5m 23s
16:	learn: 1.0865698	total: 5.58s	remaining: 5m 22s
17:	learn: 1.0816427	total: 5.93s	remaining: 5m 23s
18:	learn: 1.0767731	total: 6.2s	remaining: 5m

In [69]:
df['cbow_1_boost_predict'] = clf_1.predict(np.array(df['cbow_1_vector'].tolist())).reshape((-1))

Тоже смотрим качество

In [70]:
accuracy_score(
    y_true=df.loc[df['sample_part'] == 'test', TARGET],
    y_pred=df.loc[df['sample_part'] == 'test', 'cbow_1_boost_predict']
)

0.6062083634320572

Уже лучше, но все равно не то качество

In [71]:
pickle.dump(pipe, open('/kaggle/working/cbow_1_boost.pkl', 'wb'))

In [72]:
run = wandb.init(project='aaa w2v experiment')
artifact = wandb.Artifact(name='cbow_1_boost', type='data')
artifact.add_file('/kaggle/working/cbow_1_boost.pkl')
run.log_artifact(artifact)
run.finish()

## skip-gram

In [73]:
sg_model_1 = gensim.models.Word2Vec(
    sentences=train_sentences,
    vector_size=256, # увеличиваем размерность вектора
    alpha=0.025,
    window=5,
    min_count=10,
    seed=RANDOM_SEED,
    workers=16,
    min_alpha=0.0001,
    sg=1,
    hs=0,
    cbow_mean=1,
    compute_loss=True,
    callbacks=[log_callback()],
    epochs=50,
    null_word=0
)

Loss after epoch 0: 10597644.0
Loss after epoch 1: 18933054.0
Loss after epoch 2: 25663268.0
Loss after epoch 3: 32913638.0
Loss after epoch 4: 37852104.0
Loss after epoch 5: 42952596.0
Loss after epoch 6: 47936392.0
Loss after epoch 7: 52864984.0
Loss after epoch 8: 57679624.0
Loss after epoch 9: 62701128.0
Loss after epoch 10: 67177144.0
Loss after epoch 11: 67813336.0
Loss after epoch 12: 68460432.0
Loss after epoch 13: 69096712.0
Loss after epoch 14: 69729208.0
Loss after epoch 15: 70374344.0
Loss after epoch 16: 70998408.0
Loss after epoch 17: 71622744.0
Loss after epoch 18: 72244296.0
Loss after epoch 19: 72857240.0
Loss after epoch 20: 73464816.0
Loss after epoch 21: 74059912.0
Loss after epoch 22: 74682592.0
Loss after epoch 23: 75285312.0
Loss after epoch 24: 75892336.0
Loss after epoch 25: 76475400.0
Loss after epoch 26: 77075512.0
Loss after epoch 27: 77646008.0
Loss after epoch 28: 78245256.0
Loss after epoch 29: 78781176.0
Loss after epoch 30: 79331592.0
Loss after epoch 3

In [102]:
df['sg_1_vector'] = df['description'].apply(lambda sent: get_avg_vector(sent, sg_model_1))

### Фитим логрег

In [103]:
pipe_sg = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(n_jobs=-1, max_iter=500, random_state=RANDOM_SEED))
])

pipe_sg.fit(
    X=np.array(df.loc[df['sample_part'] == 'train', 'sg_1_vector'].tolist()),
    y=df.loc[df['sample_part'] == 'train', TARGET].values
)

Pipeline(steps=[('scaler', StandardScaler()),
                ('lr',
                 LogisticRegression(max_iter=500, n_jobs=-1, random_state=42))])

In [104]:
df['sg_1_lr_predict'] = pipe_sg.predict(np.array(df['sg_1_vector'].tolist()))

Качество

In [105]:
accuracy_score(
    y_true=df.loc[df['sample_part'] == 'test', TARGET],
    y_pred=df.loc[df['sample_part'] == 'test', 'sg_1_lr_predict']
)

0.6029175696283811

Уже лучше, чем в прошлый раз. Скорее всего, из-за увеличения размерности векторов

In [106]:
pickle.dump(pipe_sg, open('/kaggle/working/sg_1_lr.pkl', 'wb'))

In [113]:
run = wandb.init(project='aaa w2v experiment')
artifact = wandb.Artifact(name='sg_1_logreg', type='data')
artifact.add_file('/kaggle/working/sg_1_lr.pkl')
run.log_artifact(artifact)
run.finish()

wandb: ERROR Control-C detected -- Run data was not synced


### Бустинг

In [115]:
# сначала фитим катбуст с дефолтными параметрами
clf_sg = CatBoostClassifier(**BASE_CATBOOST_PARAMS, verbose=True)

clf_sg.fit(
    X=np.array(df.loc[df['sample_part'] == 'train', 'sg_1_vector'].tolist()),
    y=df.loc[df['sample_part'] == 'train', TARGET].values,
    verbose=True
)

Learning rate set to 0.100992
0:	learn: 1.3407052	total: 671ms	remaining: 11m 10s
1:	learn: 1.3018490	total: 1.29s	remaining: 10m 46s
2:	learn: 1.2700299	total: 1.94s	remaining: 10m 43s
3:	learn: 1.2441421	total: 2.62s	remaining: 10m 52s
4:	learn: 1.2212439	total: 3.29s	remaining: 10m 55s
5:	learn: 1.2000031	total: 3.93s	remaining: 10m 50s
6:	learn: 1.1830245	total: 4.6s	remaining: 10m 52s
7:	learn: 1.1669964	total: 5.2s	remaining: 10m 44s
8:	learn: 1.1534508	total: 5.79s	remaining: 10m 38s
9:	learn: 1.1418864	total: 6.43s	remaining: 10m 37s
10:	learn: 1.1304045	total: 7.12s	remaining: 10m 40s
11:	learn: 1.1199199	total: 7.74s	remaining: 10m 36s
12:	learn: 1.1116092	total: 8.35s	remaining: 10m 33s
13:	learn: 1.1035893	total: 8.97s	remaining: 10m 31s
14:	learn: 1.0965341	total: 9.58s	remaining: 10m 29s
15:	learn: 1.0902049	total: 10.2s	remaining: 10m 26s
16:	learn: 1.0842548	total: 10.9s	remaining: 10m 27s
17:	learn: 1.0787731	total: 11.5s	remaining: 10m 25s
18:	learn: 1.0735943	total: 

In [128]:
df['sg_1_boost_predict'] = clf_sg.predict(np.array(df['sg_1_vector'].tolist())).reshape(-1)

In [129]:
# качество
accuracy_score(
    y_true=df.loc[df['sample_part'] == 'test', TARGET],
    y_pred=df.loc[df['sample_part'] == 'test', 'sg_1_boost_predict']
)

0.6194718677261417

Лучше, но до TF-IDF все равно не дотягивает

Сохраняем модели

In [130]:
pickle.dump(pipe, open('/kaggle/working/sg_1_boost.pkl', 'wb'))

In [ ]:
run = wandb.init(project='aaa w2v experiment')
artifact = wandb.Artifact(name='sg_1_boost', type='data')
artifact.add_file('/kaggle/working/sg_1_boost.pkl')
run.log_artifact(artifact)
run.finish()

И на последок сохраним модели W2V

In [132]:
pickle.dump(cbow_model_1, open('/kaggle/working/cbow_model_1.pkl', 'wb'))
pickle.dump(sg_model_1, open('/kaggle/working/sg_model_1.pkl', 'wb'))